In [1]:
from vplants.autowig import autowig

In [2]:
from path import path

rootdir = path('../..').abspath()

includedir = rootdir + "/stat_tool/build-scons/include"
headers = [str(f) for f in includedir.walkfiles('*.h')]

flags = ['-x', 'c++', '-g', '-std=c++11', '-stdlib=libstdc++',
         '-I/usr/include', '-I' + str(rootdir + '/tool/build-scons/include'), '-I' + str(includedir), '-D__STDC_CONSTANT_MACROS',
         '-I/usr/local/lib/clang/3.7.0/include', '-D__STDC_LIMIT_MACROS']

force = True

asg = autowig.AbstractSemanticGraph()
autowig.front_end.plugin = 'pyclanglite'
autowig.front_end(asg, headers, flags=flags, silent=True,
                  force_overload=True, bootstrap=3, cache=rootdir + '/.AutoWIG', force=force)

In [3]:
asg['::std::ostringstream'].clean = False
asg['enum ::std::_Ios_Openmode'].clean = False
for cst in asg['enum ::std::_Ios_Openmode'].constants:
    cst.clean = False
asg.clean()

In [4]:
autowig.back_end.plugin = 'boost_python:std_filter'
autowig.back_end(asg)

from vplants.autowig.boost_python_back_end import BoostPythonExportMappingFileProxy

autowig.back_end.plugin = 'boost_python:export'
autowig.back_end(asg,
                 directory = rootdir + '/stat_tool/src/wrapper',
                 pattern = '.*stat_tool.*',
                 proxy = BoostPythonExportMappingFileProxy,
                 prefix = '_')

autowig.back_end(asg,
                 directory = rootdir + '/stat_tool/src/wrapper',
                 pattern = '(::std::ostringstream|enum ::std::_Ios_Openmode)',
                 prefix = '_')

autowig.back_end.plugin = 'boost_python:closure'
autowig.back_end(asg)

autowig.back_end.plugin = 'boost_python:export'
autowig.back_end(asg,
                 directory = rootdir + '/stat_tool/src/wrapper',
                 pattern = '.*',
                 proxy = BoostPythonExportMappingFileProxy,
                 prefix = '_')

autowig.back_end.plugin = 'boost_python:module'
autowig.back_end(asg,
                 filename = rootdir + '/stat_tool/src/wrapper/__stat_tool.cpp',
                 package = 'openalea.stat_tool')

autowig.back_end.plugin = 'boost_python:import'
autowig.back_end(asg,
                 filename = rootdir + '/stat_tool/src/openalea/stat_tool/_stat_tool.py',
                 module = rootdir + '/stat_tool/src/wrapper/__stat_tool.cpp')

In [5]:
autowig.back_end.plugin = 'on_disk'
autowig.back_end(asg,
                 pattern = rootdir + '/(stat_tool|misc)/src/(wrapper|openalea)/.*')

In [6]:
autowig.scons(rootdir + '/stat_tool', '-j3')